# Прогнозирование. Модели.

> 🚀 В этой практике нам понадобятся: `etna==2.10.0, numpy==1.26.4, pandas==1.5.3, matplotlib==3.10.3, seaborn==0.13.2, lightgbm==4.6.0, prophet==1.1.6` 

> 🚀 Установить вы их можете с помощью команды: `%pip install etna==2.10.0 numpy==1.26.4 pandas==1.5.3 matplotlib==3.10.3 seaborn==0.13.2 lightgbm==4.6.0 prophet==1.1.6` 


## Содержание

* [Загрузка выборок](#Загрузка-выборок)
* [Статистические модели](#Статистические-модели)
* [Модели регрессоры](#Модели-регрессоры)
  * [Генерация признаков](#Генерация-признаков)
  * [Линейная регрессия](#Линейная-регрессия)
  * [LGBM : Бустинг](#LGBM-:-Бустинг)
* [Заключение](#Заключение)
* [Вопросы для закрепления](#Вопросы-для-закрепления)


Привет! В предыдущем ноутбуке мы подготовили выборки, на которых теперь будем учить и тестировать модели. 

In [87]:
from datetime import date, timedelta
from pathlib import Path

import warnings
warnings.filterwarnings("ignore")

import ipywidgets as widgets
from matplotlib import pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")

from etna.datasets.tsdataset import TSDataset
from etna.models import HoltWintersModel
from etna.pipeline import Pipeline

import numpy as np 
import pandas as pd 

from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression
from prophet.make_holidays import make_holidays_df

## Загрузка выборок

Для начала загрузим ранее подготовленные выборки.

In [88]:
dpath = Path().cwd() / "ts_datasets"

train_fpath = dpath / "train.csv"
train_df = pd.read_csv(train_fpath, parse_dates=["timestamp"], index_col=0)

test_fpath = dpath / "test.csv"
test_df = pd.read_csv(test_fpath, parse_dates=["timestamp"], index_col=0)

train_df.shape, test_df.shape

((222288, 5), (2728, 5))

В дальнейшем нам также понадобится информация о: 
- горизонте прогнозирования (в нашем случае хотим на 31 день);
- дата разделения данных (1 декабря 2016г).

Вынесем их в отдельные переменные, чтобы меньше хардкодить. 

In [89]:
HORIZON = 31
SPLIT_DATE = date(2016, 12, 1)

## Статистические модели

Начнём со статистических моделей. Таким моделям по сути не нужно обучение, т.к. они используют чисто статистические зависимости, на базе которых генерируют предсказания в будущее. 

В этот раз мы посмотрим на модель Хольта-Уинтерса. Звучит пафосно, но под красивым названиям скрывается экспоненциальное сглаживание. 

Эта модель уже есть в ETNA, давайте ею и воспользуемся.

Так как работаем с ETNA, снова конвертируем таблицы в ETNA-датасеты и имитируем `segment` для нашего временного ряда. Сделаем это как в обучающей выборке, так и в тестовой. 

In [90]:
train_ts_df = train_df.copy(deep=True)
test_ts_df = test_df.copy(deep=True)

In [91]:
# TODO - 添加segment列
train_ts_df["segment"] = train_ts_df["country"] + "_" + train_ts_df["store"] + "_" + train_ts_df["product"]
test_ts_df["segment"] = test_ts_df["country"] + "_" + test_ts_df["store"] + "_" + test_ts_df["product"]


In [92]:
assert "segment" in train_ts_df.columns, "Колонка `segment` не найдена"
assert "segment" in test_ts_df.columns, "Колонка `segment` не найдена"

print("Тесты прошли! Всё хорошо!")

Тесты прошли! Всё хорошо!


In [93]:
# TODO - 删除列country, store和product
train_ts_df = train_ts_df.drop(columns=["country", "store", "product"])
test_ts_df = test_ts_df.drop(columns=["country", "store", "product"])

In [94]:
train_cols = train_ts_df.columns
test_cols = test_ts_df.columns 

for col_name in ["country", "store", "product"]:
    assert col_name not in train_cols, f"В обучающей выборке всё ещё есть колонка {col_name}"
    assert col_name not in test_cols, f"В тестовой выборке всё ещё есть колонка {col_name}"

print("Тесты прошли! Всё хорошо!")

Тесты прошли! Всё хорошо!


In [95]:
# TODO - 转换pandas表格为ETNA Dataset
train_ts_df = TSDataset(df=train_ts_df, freq="D")
test_ts_df = TSDataset(df=test_ts_df, freq="D")

In [96]:
assert isinstance(train_ts_df, TSDataset), f"Неверный тип данных: {type(train_ts_df)}"
assert isinstance(test_ts_df, TSDataset), f"Неверный тип данных: {type(test_ts_df)}"

print("Тесты прошли! Всё хорошо!")

Тесты прошли! Всё хорошо!


Датасеты сконвертированы, можно "учить" модель. 

Но на поверхности лежит вопрос: "А на чём учить, если признаков-то нет?". И это хороший вопрос. ETNA позволяет сгенерировать фичи прямо через интерфейс обучения модели.

В нашем случае добавим: 
- информацию о тренде в данных;
- информацию о сезонной составляющей.

<details>
    <summary>🤓 Напоминалка про тренды и сезонность [Нажми на меня]</summary>

**Тренд** - это какая-то общая зависимость, характерная для всего ряда целиком. **Сезонность** - это периодически повторяющаяся компонента. Обе эти составляющие часто присутствуют в ряде одновременно. 

</details>

In [97]:
model = HoltWintersModel(
    trend="add", 
    seasonal="add", 
    seasonal_periods=7,
)
pipeline = Pipeline(model=model, horizon=HORIZON)

pipeline.fit(train_ts_df)

Pipeline(model = HoltWintersModel(trend = 'add', damped_trend = False, seasonal = 'add', seasonal_periods = 7, initialization_method = 'estimated', initial_level = None, initial_trend = None, initial_seasonal = None, use_boxcox = False, bounds = None, dates = None, freq = None, missing = 'none', smoothing_level = None, smoothing_trend = None, smoothing_seasonal = None, damping_trend = None, ), transforms = (), horizon = 31, )

Модель обучили, теперь можно и предсказания получить. 

Немного поиграемся с переименование колонок, чтобы дальше было удобнее считать метрики

In [98]:
# получаем предсказания 
ts_forecast_df = pipeline.forecast()

# в тестовой выборке целевую переменную переименуем в y_test (это true значения)
forecast_test_df = test_ts_df.to_pandas(flatten=True).rename(columns={"target": "y_test"})

# в датасете с предсказания целевую переменную переименуем в y_pred (это предсказанные значения)
forecast_df = ts_forecast_df.to_pandas(flatten=True).rename(columns={"target": "y_pred"})

# слепим таблицы, чтобы получить общие результаты
hotl_winters_predictions = forecast_test_df.merge(
    forecast_df, 
    on=["timestamp", "segment"], 
    how="outer"
)

# разделяем segment обратно на 3 составляющие 
hotl_winters_predictions["country"] = hotl_winters_predictions["segment"].str.split("_").str[0]
hotl_winters_predictions["store"] = hotl_winters_predictions["segment"].str.split("_").str[1]
hotl_winters_predictions["product"] = hotl_winters_predictions["segment"].str.split("_").str[2]
hotl_winters_predictions = hotl_winters_predictions.drop(columns=["segment"])

# округляем предсказания от модели, т.к. продавать часть пышки - не вариант 
hotl_winters_predictions["y_pred"] = np.ceil(hotl_winters_predictions["y_pred"])#.astype(int)

hotl_winters_predictions.head()

,timestamp,y_test,y_pred,country,store,product
0,2016-12-01,NaN,215.0,Canada,Donots,Chocolate
1,2016-12-02,212.0,219.0,Canada,Donots,Chocolate
2,2016-12-03,210.0,213.0,Canada,Donots,Chocolate
3,2016-12-04,231.0,233.0,Canada,Donots,Chocolate
4,2016-12-05,NaN,215.0,Canada,Donots,Chocolate


In [99]:
hotl_winters_predictions.shape

(2728, 6)

Отрисуем полученные предсказания в сравнении с истинными значениями на графике. 

In [100]:
@widgets.interact(
    country=widgets.Dropdown(options=hotl_winters_predictions["country"].unique()),
    store=widgets.Dropdown(options=hotl_winters_predictions["store"].unique()),
    products=widgets.Dropdown(options=hotl_winters_predictions["product"].unique()),
    start_date=widgets.DatePicker(value=hotl_winters_predictions["timestamp"].min()),
)
def show_holt_winters_predictions(country: str, store: str, products: str, start_date: date):
    start_date = pd.Timestamp(start_date)
    plot_df = hotl_winters_predictions[
        (hotl_winters_predictions["country"] == country)
        & (hotl_winters_predictions["store"] == store)
        & (hotl_winters_predictions["product"] == products)
        & (hotl_winters_predictions["timestamp"].dt.date >= start_date.date())
    ]

    fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(15, 4))

    axs.plot(plot_df["timestamp"], plot_df["y_pred"], marker="o", label="pred")
    # уберём пропущенные значения, чтобы точки соединились между собой - во имя субъктивного прекрасного 
    plot_df = plot_df.dropna()
    axs.plot(plot_df["timestamp"], plot_df["y_test"], marker="o", label="true")
    axs.set_xlabel("Дата")
    axs.set_ylabel("Кол-во проданных пышек")
    axs.legend()

    plt.show()

interactive(children=(Dropdown(description='country', options=('Canada', 'Finland', 'Italy', 'Kenya', 'Norway'…

Чтобы не потерять результаты работы модели, сохраним предсказания в файл.

In [101]:
pred_dpath = Path("forecast_predictions")
pred_dpath.mkdir(parents=True, exist_ok=True)

pred_fpath = pred_dpath / "holt_winters_predictions.csv"
hotl_winters_predictions.to_csv(pred_fpath)

## Модели регрессоры

Для прогнозирования можно использовать те же модели, что и в "сырой" регрессии. Например, линейную регрессию или LGBM. Их мы и поварим для наших пышек.

Но есть большое жирное **НО**. Здесь мы уже не сможем использовать ETNA, и все признаки нужно будет сгенерировать ручками. Что ж, погнали! 

### Генерация признаков

Признаки нужны будут как для обучающей выборки, так и для тестовой. Поэтому объединим их обратно в единую таблицу, сгенерим признаки, а потом снова разделим по той же дате 1 декабря 2016. 

In [102]:
df = pd.concat((train_df, test_df)).set_index("timestamp")

df.head()

,country,store,product,target
timestamp,,,,
2010-01-01,Canada,Donots,Chocolate,300.0
2010-01-02,Canada,Donots,Chocolate,281.0
2010-01-03,Canada,Donots,Chocolate,297.0
2010-01-04,Canada,Donots,Chocolate,235.0
2010-01-05,Canada,Donots,Chocolate,240.0


Нет единого верного способа какие признаки генерить и как, тут вас ограничивает только ваша фантазия и доступные данные. 

Мы создадим три типа признаков: 
- признаки с информацией о днях;
- признаки о праздниках;
- признаки о "лагах".

<details>
    <summary>🤓 Что такое лаги? [Нажми на меня]</summary>

**Лаги** - это какие-то предыдущие значения временного ряда. Например, первый лаг - это вчерашнее значение, а пятый лаг - это значение 5 дней назад и т.д. Такие признаки очень важны для регрессионных моделей, т.к. им нужно получить информацию о прошлом ряда. 

</details>

Генерить признаки (фичи) нужно для каждого ряда в отдельности, т.к. для каждого ряда мы будем учить свою собственную модель. В итоге у нас получится 90 моделей регрессии (на самом деле 88, т.к. 2 пустых ряда мы выкинули). А не одна, как в ETNA. 

Давайте для примера посмотрим генерацию признаков для одного ряда, а потом напишем цикл (пайплайн), который будет проходить по каждому ряду. 

Допустим выберем кофейную пышку в пышечной Pyshka в Сингапуре.

In [103]:
SAMPLE_COUNTRY = "Singapore"
SAMPLE_STORE = "Pyshka"
SAMPLE_PRODUCT = "Coffee"

In [104]:
# TODO - 过滤只针对一个序列的数据
sample_df = df[
    (df["country"] == SAMPLE_COUNTRY) 
    & (df["store"] == SAMPLE_STORE) 
    & (df["product"] == SAMPLE_PRODUCT)
]

In [105]:
sample_df.head()

,country,store,product,target
timestamp,,,,
2010-01-01,Singapore,Pyshka,Coffee,1045.0
2010-01-02,Singapore,Pyshka,Coffee,1010.0
2010-01-03,Singapore,Pyshka,Coffee,1040.0
2010-01-04,Singapore,Pyshka,Coffee,846.0
2010-01-05,Singapore,Pyshka,Coffee,770.0


In [106]:
assert set(sample_df["country"].unique()) == set([SAMPLE_COUNTRY]), "Должен быть только Сингапур в данных"
assert set(sample_df["store"].unique()) == set([SAMPLE_STORE]), "Должна быть только одна пышечная в данных"
assert set(sample_df["product"].unique()) == set([SAMPLE_PRODUCT]), "Должна быть только кофейная пышка в данных"

print("Тесты прошли! Всё хорошо!")

Тесты прошли! Всё хорошо!


Оставим только информацию о дате и целевую колонку. Остальные колонки содержат повторяющиеся значения и модели они ничем не помогут.

In [107]:
sample_df = sample_df[["target"]]
sample_df.head()

,target
timestamp,
2010-01-01,1045.0
2010-01-02,1010.0
2010-01-03,1040.0
2010-01-04,846.0
2010-01-05,770.0


Начнём с информации о днях. Сюда относится такая информация как день месяца, день недели, четверть года, к которому принадлежит текущий timestamp, порядковый день года, порядковая неделя года и т.д и т.п. Всё, что вы только можете придумать.  

Давайте сделаем отдельную функцию для генерации таких фич, она нам пригодится ещё в пайплайне.

In [108]:
def create_dtime_features(df: pd.DataFrame) -> pd.DataFrame:
    """Create day-time features."""
    df_new = df.copy()

    df_new["dayofweek"] = df_new.index.dayofweek.tolist()
    df_new["cos_weekday"] = np.cos(df_new["dayofweek"] / 7 * 2 * np.pi)
    df_new["sin_weekday"] = np.sin(df_new["dayofweek"] / 7 * 2 * np.pi)

    df_new["is_leap_year"] = df_new.index.is_leap_year.tolist()
    df_new["dayofyear"] = df_new.index.dayofyear.tolist()
    df_new["cos_doy"] = np.cos(df_new["dayofyear"] / (365 + df_new["is_leap_year"]) * 2 * np.pi)
    df_new["sin_doy"] = np.sin(df_new["dayofyear"] / (365 + df_new["is_leap_year"]) * 2 * np.pi)

    df_new["quarter"] = df_new.index.quarter.tolist()
    df_new["year"] = df_new.index.year.tolist()
    df_new["month"] = df_new.index.month.tolist()
    df_new["day"] = df_new.index.day.tolist()
    df_new["weekofyear"] = df_new.index.isocalendar().week.tolist()

    df_new["is_weekend"] = 0
    df_new.loc[df_new["dayofweek"].isin([5, 6]), "is_weekend"] = 1

    df_new["is_month_start"] = df_new.index.is_month_start.tolist()
    df_new["is_month_end"] = df_new.index.is_month_end.tolist()

    return df_new

In [109]:
# Вызываем функцию и посмотрим, что у нас стало с табличкой
sample_df = create_dtime_features(sample_df)

sample_df.head()

,target,dayofweek,cos_weekday,sin_weekday,is_leap_year,dayofyear,cos_doy,sin_doy,quarter,year,month,day,weekofyear,is_weekend,is_month_start,is_month_end
timestamp,,,,,,,,,,,,,,,,
2010-01-01,1045.0,4,-0.900969,-0.433884,False,1,0.999852,0.017213,1,2010,1,1,53,0,True,False
2010-01-02,1010.0,5,-0.222521,-0.974928,False,2,0.999407,0.034422,1,2010,1,2,53,1,False,False
2010-01-03,1040.0,6,0.623490,-0.781831,False,3,0.998667,0.051620,1,2010,1,3,53,1,False,False
2010-01-04,846.0,0,1.000000,0.000000,False,4,0.997630,0.068802,1,2010,1,4,1,0,False,False
2010-01-05,770.0,1,0.623490,0.781831,False,5,0.996298,0.085965,1,2010,1,5,1,0,False,False


Пуф! И у нас уже 15 новых признаков (колонок) 😎 Дальше больше! 

Теперь добавим признаки о праздниках. Есть радостная новость, праздники не нужно добавлять руками, за нас это уже сделали другие люди в библиотеке `Prophet`. 

Достаточно указать страну и диапазон лет. И получим исчерпывающий список праздников.

Для удобства модели закодируем бинарным значением каждый праздник.

In [110]:
def get_holiday_df(country_name: str) -> pd.DataFrame:
    holidays_df = make_holidays_df(year_list=range(2010, 2017), country=country_name)

    holidays_df["is_holiday"] = 1
    holidays_df = holidays_df.rename(columns={"ds": "timestamp"})
    holidays_df = holidays_df.set_index("timestamp")

    return holidays_df

In [111]:
sample_holidays_df = get_holiday_df(SAMPLE_COUNTRY)

sample_holidays_df.head()

,holiday,is_holiday
timestamp,,
2016-01-01,New Year's Day,1
2016-02-08,Chinese New Year,1
2016-02-09,Chinese New Year,1
2016-07-06,Eid al-Fitr,1
2016-09-12,Eid al-Adha,1


Чтобы расширить признаки, добавим в таблицу лаги после праздничного дня (1 и 2 дня после).

Для удобства оформим всё это в отдельную функцию.

In [112]:
def create_holiday_features(df: pd.DataFrame, holidays_df: pd.DataFrame) -> pd.DataFrame:
    """Create holiday features."""
    df_new = df.copy()
    df_new = df_new.merge(holidays_df[["is_holiday"]], left_index=True, right_index=True, how="outer")
    df_new["is_holiday"] = df_new["is_holiday"].fillna(0)
    df_new["holiday_lag_1"] = df_new["is_holiday"].shift(1)
    df_new["holiday_lag_2"] = df_new["is_holiday"].shift(2)
    return df_new

In [113]:
sample_df = create_holiday_features(sample_df, sample_holidays_df)

sample_df.head()

,target,dayofweek,cos_weekday,sin_weekday,is_leap_year,dayofyear,cos_doy,sin_doy,quarter,year,month,day,weekofyear,is_weekend,is_month_start,is_month_end,is_holiday,holiday_lag_1,holiday_lag_2
timestamp,,,,,,,,,,,,,,,,,,,
2010-01-01,1045.0,4,-0.900969,-0.433884,False,1,0.999852,0.017213,1,2010,1,1,53,0,True,False,1.0,NaN,NaN
2010-01-02,1010.0,5,-0.222521,-0.974928,False,2,0.999407,0.034422,1,2010,1,2,53,1,False,False,0.0,1.0,NaN
2010-01-03,1040.0,6,0.623490,-0.781831,False,3,0.998667,0.051620,1,2010,1,3,53,1,False,False,0.0,0.0,1.0
2010-01-04,846.0,0,1.000000,0.000000,False,4,0.997630,0.068802,1,2010,1,4,1,0,False,False,0.0,0.0,0.0
2010-01-05,770.0,1,0.623490,0.781831,False,5,0.996298,0.085965,1,2010,1,5,1,0,False,False,0.0,0.0,0.0


Ещё +3 признака. Итого уже 18 признаков для обучения модели.

Переходим к "лагнутым" признакам. Сгенерим лаги на 7, 10 и 15 дней назад. 

Помимо этого добавим ещё немного статистики по лагам: скользящее среднее значение с окном 14 и 7 дней. 

Опять же какие лаги выбрать, чтобы модели помочь, никто не скажет. Тут нужно экспериментировать и отталкиваться от задачи и данных.

Для удобства обернём всё в функцию.

In [114]:
def create_lag_features(df: pd.DataFrame, horizon) -> pd.DataFrame:
    lags = [7, 10, 15]
    df_new = df.copy()
    
    for lag in lags: 
        for lag_value in range(horizon, horizon + lag):
            feature_name = f"lag_{lag_value}"
            df_new[feature_name] = df_new["target"].shift(lag)

            df_new[f"{feature_name}_rolling_mean_30"] = df_new[feature_name].rolling(14).mean() 
            df_new[f"{feature_name}_rolling_mean_7"] = df_new[feature_name].rolling(7).mean()

    return df_new

In [115]:
sample_df = create_lag_features(sample_df, horizon=HORIZON)
sample_df.head()

,target,dayofweek,cos_weekday,sin_weekday,is_leap_year,dayofyear,cos_doy,sin_doy,quarter,year,...,lag_42_rolling_mean_7,lag_43,lag_43_rolling_mean_30,lag_43_rolling_mean_7,lag_44,lag_44_rolling_mean_30,lag_44_rolling_mean_7,lag_45,lag_45_rolling_mean_30,lag_45_rolling_mean_7
timestamp,,,,,,,,,,,,,,,,,,,,,
2010-01-01,1045.0,4,-0.900969,-0.433884,False,1,0.999852,0.017213,1,2010,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-02,1010.0,5,-0.222521,-0.974928,False,2,0.999407,0.034422,1,2010,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-03,1040.0,6,0.623490,-0.781831,False,3,0.998667,0.051620,1,2010,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-04,846.0,0,1.000000,0.000000,False,4,0.997630,0.068802,1,2010,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-05,770.0,1,0.623490,0.781831,False,5,0.996298,0.085965,1,2010,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [116]:
sample_df.tail()

,target,dayofweek,cos_weekday,sin_weekday,is_leap_year,dayofyear,cos_doy,sin_doy,quarter,year,...,lag_42_rolling_mean_7,lag_43,lag_43_rolling_mean_30,lag_43_rolling_mean_7,lag_44,lag_44_rolling_mean_30,lag_44_rolling_mean_7,lag_45,lag_45_rolling_mean_30,lag_45_rolling_mean_7
timestamp,,,,,,,,,,,,,,,,,,,,,
2016-12-27,1079.0,1,0.623490,0.781831,True,362,0.997643,-6.861474e-02,4,2016,...,978.571429,895.0,987.928571,978.571429,895.0,987.928571,978.571429,895.0,987.928571,978.571429
2016-12-28,1050.0,2,-0.222521,0.974928,True,363,0.998674,-5.147875e-02,4,2016,...,956.571429,796.0,976.642857,956.571429,796.0,976.642857,956.571429,796.0,976.642857,956.571429
2016-12-29,1133.0,3,-0.900969,0.433884,True,364,0.999411,-3.432760e-02,4,2016,...,962.142857,958.0,976.142857,962.142857,958.0,976.142857,962.142857,958.0,976.142857,962.142857
2016-12-30,1253.0,4,-0.900969,-0.433884,True,365,0.999853,-1.716633e-02,4,2016,...,942.571429,858.0,965.642857,942.571429,858.0,965.642857,942.571429,858.0,965.642857,942.571429
2016-12-31,1242.0,5,-0.222521,-0.974928,True,366,1.000000,-2.449294e-16,4,2016,...,925.857143,937.0,962.071429,925.857143,937.0,962.071429,925.857143,937.0,962.071429,925.857143


Важно отметить, что в начале нашей выборки часть лагнутых признаков содержит `NaN` в ячейках. Это случилось потому, что это начало истории и лаги просто неоткуда взять. При обучении эти строки придётся дропнуть.

Но так как это начало обучающей выборки, то не страшно. Тем более уйдёт всего около 30 строчек, что меньше 1% процента наших данных.

In [117]:
sample_df.shape

(2557, 64)

Итак ... лёгким движеним руки у нас уже 63 признака. А помните с чего мы начинали? 1 колонка с днём и 1 целевая колонка.  

Можно добавить и больше признаков, но пока остановимся на этом.

Тут как и везде важно **не переборщить**. Когда признаков очень много (больше 100), то моделям уже сложно разделить их в пространстве и эффективность обучения снижается. 

Если бы у нас был всего 1 ряд, мы бы полученную таблицу разделили обратно на две выборки и обучили бы 1 модель. 

Но так как у нас есть ещё 87 рядов, то давайте напишем цикл, в котором для каждого ряда повторим эти шаги, обучим модель и получим предсказания.

In [118]:
# функция для форматирования полученных предсказаний, чтобы формат таблиц был такой же как у ETNA 
def create_preds_df(y_test, y_pred, country, store, product):
    y_test = y_test.rename(columns={"target": "y_test"})

    y_test["y_pred"] = y_pred
    # округляем предсказания, пышки по частям мы не продаём
    y_test["y_pred"] = np.ceil(y_test["y_pred"])

    y_test["country"] = country
    y_test["store"] = store
    y_test["product"] = product
    preds_df = y_test.reset_index()
    preds_df = preds_df[["timestamp", "y_test", "y_pred", "country", "store", "product"]]

    return preds_df

Собираем всё в большую и красивую кучку.

In [119]:
def run_pipeline(model) -> pd.DataFrame:
    # выделяем уникальные страны в данных 
    countries = df["country"].unique()

    # выделяем уникальные пышечные в данных
    stores = df["store"].unique()

    # выделяем уникальные вкусы пышек в данных
    products = df["product"].unique()

    # кол-во комбинаций, чисто для красивых логов 
    iter_n = len(countries) * len(stores) * len(products)

    missing_predictions = []
    predictions = []

    iter_counter = 1

    # итерируемся по всем комбинациям страна - пышечная - вкус 
    for country in countries:
        for store in stores:
            for product in products:
                print(f"{iter_counter}/{iter_n} {country!r} - {store!r} - {product!r} processing ...")

                # выбираем только 1 временной ряд
                sample_df = df[
                    (df["country"] == country) 
                    & (df["store"] == store)
                    & (df["product"] == product)
                ]
                nan_target_index = sample_df[sample_df["target"].isna()].index
                sample_df = sample_df[["target"]].reset_index()

                if sample_df.empty:
                    missing_predictions.append((country, store, product))
                    continue
                
                sample_df = sample_df.resample("1D", on="timestamp").sum()
                sample_df.index = pd.to_datetime(sample_df.index)

                # генерим временные признаки
                sample_df = create_dtime_features(sample_df)

                # генерим праздничные признаки 
                sample_holidays_df = get_holiday_df(country)
                sample_df = create_holiday_features(sample_df, sample_holidays_df)
                
                # генерим лагнутые признаки 
                sample_df = create_lag_features(sample_df, horizon=HORIZON)

                # делим данные обратно на 2 выборки: train и test

                # обучающая выборка
                sample_train_df = sample_df.loc[: SPLIT_DATE - timedelta(days=1)]
                # дропнем строки, где есть пропущенные значения 
                sample_train_df = sample_train_df.dropna()

                # тестовая выборка
                sample_test_df = sample_df.loc[SPLIT_DATE:]

                # разделяем выборки на признаки и целевую переменную
                X_train = sample_train_df.drop(columns=["target"])
                y_train = sample_train_df["target"]

                X_test = sample_test_df.drop(columns=["target"])
                # X_test = X_test.fillna(method="bfill").fillna(method="ffill")

                y_test = sample_test_df[["target"]]
                nan_y_test_index = set(y_test.index) & set(nan_target_index)
                y_test.loc[nan_y_test_index, "target"] = None

                iter_counter += 1

                # на всякий случай проверяем данные на "пустоту"
                if X_train.empty or X_test.empty:
                    missing_predictions.append((country, store, product))
                    continue

                # обучаем модель
                model.fit(X_train, y_train)
                # получаем предсказания
                y_pred = model.predict(X_test)

                # форматируем предсказания в удобный нам формат
                preds_df = create_preds_df(y_test, y_pred, country, store, product)
                predictions.append(preds_df)

    # объединяем предсказания ото всех рядов в единую таблицу
    predictions_df = pd.concat(predictions)
    return predictions_df

Все приготовления сделаны, погнали учить модельки! 

### Линейная регрессия

Первой будет старая добрая модель линейной регрессии.

In [120]:
linreg_model = LinearRegression()
linreg_predictions = run_pipeline(linreg_model)

linreg_predictions.head()

1/90 'Canada' - 'Donots' - 'Chocolate' processing ...
2/90 'Canada' - 'Donots' - 'Coffee' processing ...
3/90 'Canada' - 'Donots' - 'Cream' processing ...
4/90 'Canada' - 'Donots' - 'Strawberry Jam' processing ...
5/90 'Canada' - 'Donots' - 'Sugar Powder' processing ...
6/90 'Canada' - 'Fluffy Bounce' - 'Chocolate' processing ...
6/90 'Canada' - 'Fluffy Bounce' - 'Coffee' processing ...
7/90 'Canada' - 'Fluffy Bounce' - 'Cream' processing ...
8/90 'Canada' - 'Fluffy Bounce' - 'Strawberry Jam' processing ...
9/90 'Canada' - 'Fluffy Bounce' - 'Sugar Powder' processing ...
10/90 'Canada' - 'Pyshka' - 'Chocolate' processing ...
11/90 'Canada' - 'Pyshka' - 'Coffee' processing ...
12/90 'Canada' - 'Pyshka' - 'Cream' processing ...
13/90 'Canada' - 'Pyshka' - 'Strawberry Jam' processing ...
14/90 'Canada' - 'Pyshka' - 'Sugar Powder' processing ...
15/90 'Finland' - 'Donots' - 'Chocolate' processing ...
16/90 'Finland' - 'Donots' - 'Coffee' processing ...
17/90 'Finland' - 'Donots' - 'Cream' p

,timestamp,y_test,y_pred,country,store,product
0,2016-12-01,NaN,224.0,Canada,Donots,Chocolate
1,2016-12-02,212.0,226.0,Canada,Donots,Chocolate
2,2016-12-03,210.0,231.0,Canada,Donots,Chocolate
3,2016-12-04,231.0,242.0,Canada,Donots,Chocolate
4,2016-12-05,NaN,225.0,Canada,Donots,Chocolate


In [121]:
linreg_predictions.shape

(2728, 6)

Также как с моделью ETNA отрисуем на графике сравнение целевых значений и предсказанных моделью.

In [122]:
@widgets.interact(
    country=widgets.Dropdown(options=linreg_predictions["country"].unique()),
    store=widgets.Dropdown(options=linreg_predictions["store"].unique()),
    products=widgets.Dropdown(options=linreg_predictions["product"].unique()),
    start_date=widgets.DatePicker(value=linreg_predictions["timestamp"].min()),
)
def show_linreg_predictions(country: str, store: str, products: str, start_date: date):  # noqa: D103
    start_date = pd.Timestamp(start_date)
    plot_df = linreg_predictions[
        (linreg_predictions["country"] == country)
        & (linreg_predictions["store"] == store)
        & (linreg_predictions["product"] == products)
        & (linreg_predictions["timestamp"].dt.date >= start_date.date())
    ]

    fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(15, 4))

    axs.plot(plot_df["timestamp"], plot_df["y_pred"], marker="o", label="pred")
    # уберём пропущенные значения, чтобы точки соединились между собой - чисто для красоты
    plot_df = plot_df.dropna()
    axs.plot(plot_df["timestamp"], plot_df["y_test"], marker="o", label="true")
    axs.set_xlabel("Дата")
    axs.set_ylabel("Кол-во проданных пышек")
    axs.legend()

    plt.show()

interactive(children=(Dropdown(description='country', options=('Canada', 'Finland', 'Italy', 'Kenya', 'Norway'…

И сохраняем в файл, чтобы не потерять.

In [123]:
pred_dpath = Path("forecast_predictions")
pred_dpath.mkdir(parents=True, exist_ok=True)

pred_fpath = pred_dpath / "linear_regression_predictions.csv"
linreg_predictions.to_csv(pred_fpath)

### LGBM : Бустинг

Для сравнения обучим модель градиентного бустинга.

In [124]:
lgbm_model = LGBMRegressor(random_state=42)
lgbm_predictions = run_pipeline(lgbm_model)

lgbm_predictions.head()

1/90 'Canada' - 'Donots' - 'Chocolate' processing ...
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001058 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 12377
[LightGBM] [Info] Number of data points in the train set: 2498, number of used features: 63
[LightGBM] [Info] Start training from score 223.421157
2/90 'Canada' - 'Donots' - 'Coffee' processing ...
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000898 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 12377
[LightGBM] [Info] Number of data points in the train set: 2498, number of used features: 63
[LightGBM] [Info] Start training from score 655.552842
3/90 'Canada' - 'Donots' - 'Cream' processing ...
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001115 seconds.
You can set `force_col_wise=true` to remove the overh

,timestamp,y_test,y_pred,country,store,product
0,2016-12-01,NaN,212.0,Canada,Donots,Chocolate
1,2016-12-02,212.0,211.0,Canada,Donots,Chocolate
2,2016-12-03,210.0,210.0,Canada,Donots,Chocolate
3,2016-12-04,231.0,231.0,Canada,Donots,Chocolate
4,2016-12-05,NaN,218.0,Canada,Donots,Chocolate


In [125]:
lgbm_predictions.shape

(2728, 6)

Угадайте, что? Отрисуем предсказания. 

In [126]:
@widgets.interact(
    country=widgets.Dropdown(options=lgbm_predictions["country"].unique()),
    store=widgets.Dropdown(options=lgbm_predictions["store"].unique()),
    products=widgets.Dropdown(options=lgbm_predictions["product"].unique()),
    start_date=widgets.DatePicker(value=lgbm_predictions["timestamp"].min()),
)
def show_lgbm_predictions(country: str, store: str, products: str, start_date: date):  # noqa: D103
    start_date = pd.Timestamp(start_date)
    plot_df = lgbm_predictions[
        (lgbm_predictions["country"] == country)
        & (lgbm_predictions["store"] == store)
        & (lgbm_predictions["product"] == products)
        & (lgbm_predictions["timestamp"].dt.date >= start_date.date())
    ]

    fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(15, 4))

    axs.plot(plot_df["timestamp"], plot_df["y_pred"], marker="o", label="pred")
    # уберём пропущенные значения, чтобы точки соединились между собой - чисто для красоты
    plot_df = plot_df.dropna()
    axs.plot(plot_df["timestamp"], plot_df["y_test"], marker="o", label="true")
    axs.set_xlabel("Дата")
    axs.set_ylabel("Кол-во проданных пышек")
    axs.legend()

    plt.show()

interactive(children=(Dropdown(description='country', options=('Canada', 'Finland', 'Italy', 'Kenya', 'Norway'…

Сохраняем результаты в файл.

In [127]:
pred_dpath = Path("forecast_predictions")
pred_dpath.mkdir(parents=True, exist_ok=True)

pred_fpath = pred_dpath / "lgbm_predictions.csv"
lgbm_predictions.to_csv(pred_fpath)

Ура! Модели обучены, теперь осталось самое вкусное - посчитать метрики, сравнить модели и сделать выводы. 

Этим займёмся в следующем ноутбуке. 

## Заключение

В этом ноутбуке вы познакомились со статистической моделью с пафосным названием Хольта-Винтерса. 

Посмотрели как натянуть знакомые модели регрессии на прогнозирование. 

И погрузились в удивительный мир генерации признаков. 

## Вопросы для закрепления

1. В чём разница между трендом и сезонностью? 
2. Может ли быть так, что в ряду есть тренд, но нет сезонности? 
3. Можно ли обучить регрессионную модель без лагнутых признаков? 
4. Как обучить регрессионную модель, если в данных несколько временных рядов? 
5. Что нужно сделать, чтобы обучить модель Хольта-Винтерса, если в данных несколько временных рядов? 